# MNIST - A Exploration in to Data Analysis

## 1 - Introduction

This is monologue of my progress through this project.

MNIST is the Hello World of neural networks. My aim is to learn as much as possible about neural networks and other data analysis methods by analysing this dataset as well as learning how to best use the features of the high level programming language Python.

My approach to this problem will be through the following steps:
1. Understand the raw data
2. Consider different potential solutions to the problem
3. Pick a model
4. Process the raw data so it is in a suitable form for the algorithms to work
5. Fit/train a model
6. Analyse the model and try to find improvements

### 1.1 - Understanding the Raw Data

The data are in csv files. Each file has 785 columns. For the training data, the first column is the label, that is the digit drawn by the user. The remaining 784 columns make up the images. The images are 28x28 pixels in size and each row represents one whole image. Consequently each image is of relatively low quality and is in greyscale. The testing data is identical but with the label column omitted, hence there are only 784 columns in this file.

The images can be reconstructed by taking the first 28 cells as the first row of pixels of the image. The subsequent 28 cells make up the second row of pixels of the image etc.

The submission file must have the following format:
```
ImageId,Label
1,0
2,0
3,0
etc.
```

### 1.2 - Potential Solutions

One popular solution is to use a neural network to classify the digits. This is something I will explore using Tensorflow. If this is successful I may then look to improve this training process with regards to speed and efficiency by training the model in C/C++ on my GPU.

This problem would also lend itself well to using some sort of clustering algorithm. Further research and implentation in to k-means clustering algorithms and the algorithms I have learned at university will be considered.

A classification regression tree may also work here.

Finally I am aware of support-vector machines. I am not entirely sure what these are hence more research is required, but that is another method that should be explored.

## 2 - Neural Network Model

### 2.0 - Introduction

The neural network model requires an initial layer with one node per pixel. In this case, that is 28^2 input nodes. The output layer requires as many nodes as there are classes. This final layer usually uses a softmax activation function on each of the nodes. This converts each of the nodes to a value between 0 and 1 such that the sum of all the values sums to 1. This means the output can be interpreted as a probability of the image beloging to each particular class.

The difficult part of neural networks is filling in the hidden layers. A single layer is only capable of linear regression. However, deeper networks and other special layers such as convolutional layers allow for a model to pick different features. Hidden layers will often use the rectified linear unit (ReLU) activation function on each node. This is a very natural interpretation of a neuron. 

The aim here is to learn how these layers can be combined to create a good model and this shall be done through hands on experimentation and through research.

### 2.1 - Preprocess the Data

In [1]:
# For the matplotljupyter notebook
%matplotlib inline

import numpy as np # For linear algebra
import pandas as pd # For data structures and easy file handling
import matplotlib.pyplot as plt # For plotting graphs easily
import tensorflow as tf # For neural networks using Keras

UsageError: unrecognized arguments: # For the jupyter notebook


I shall now read the train and test data files.

In [ ]:
raw_data_train = pd.read_csv('data/train.csv')
raw_data_test = pd.read_csv('data/test.csv')

print(type(raw_data_test))

print(raw_data_train.head())

print('Training dimensions', raw_data_train.shape)
print('Testing dimensions', raw_data_test.shape)

Evidently the train dataset has 42000 rows and the test dataset has 28000 rows. So there is plenty of data to be working with!

First, I need to preprocess the data so it is in a usuable form. The block below coverts the raw data stored as pd.DataFrames and converts it to np.arrays. This can be thought of as reconstructing the original image. This will be flattened after any convolutional layers (if present) in the neural network model anyway (sending it back to a 1D vector).

In [2]:
img_rows, img_cols = 28, 28
num_classes = 10

def prep_raw_data(raw_data):
    """Splits the training data in to training and validation datasets.
    """
    y = raw_data.label.values
    out_y = tf.keras.utils.to_categorical(y=y, num_classes=num_classes)
    x = raw_data.values[:, 1:]
    return out_y, reshape_data(x)

def reshape_data(array):
    """Returns n x 28 x 28 numpy array that has been standardised.
    """
    if isinstance(array, pd.DataFrame):
        #assume array is the testing data and convert it to a numpy array
        array = array.values[:,:]
        
    return array.reshape(array.shape[0], 28, 28)/255

y_train, x_train = prep_raw_data(raw_data_train)
x_test = reshape_data(raw_data_test)

NameError: name 'raw_data_train' is not defined

Just to show that the preprocessing has worked, here is a plot of a data point.

In [ ]:
plt.figure()
plt.imshow(x_train[123])
plt.colorbar()
plt.grid(False)
plt.show()

### 2.2 - First Model

Here I shall bulid a model starting with a very basic architecture and try to improve the model iteratively by adding different layer types and combinations.

First I will start with a hidden layer with 100 nodes and ReLU as the activation function. To compile to model I will just use stochastic gradient descent (SGD) at first and I shall explore other algorithms some other time. [Click here](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) to view the optimiser algorithms built in to TF. I shall set the loss function to categorical crossentropy. WHY?!?!?! The metric is added to simply show the accuracy during training, it is not required.

In [ ]:
basic_model_1 = tf.keras.models.Sequential()
basic_model_1.add(tf.keras.layers.Flatten())
basic_model_1.add(tf.keras.layers.Dense(100, activation = 'relu'))
basic_model_1.add(tf.keras.layers.Dense(num_classes, activation = 'softmax'))

basic_model_1.compile(optimizer = 'SGD', 
                    loss = tf.keras.losses.categorical_crossentropy,
                    metrics = ['accuracy'])

basic_model_1.fit(x = x_train, y = y_train)

predictions_1 = np.argmax(basic_model_1.predict(x = x_test), axis = 1)

In [ ]:
output = pd.DataFrame({'ImageId': range(1,28001), 'Label': predictions_1})
output.to_csv('Submissions.csv', index = False)

According to kaggle, this model achieved a 88.5% accuracy! Not bad. However this puts me 2906th of 3180 submissions... that's the 92nd percentile (at the time of writing, June 2020). 

The next step is to explore methods that will improve the accuracy of results. I should also change the way I check my accuracy of my models as uploading to Kaggle every time is slow.

To do this I will split the training data in to a training and validation set. I can hard code this by physically splitting the data or I can use `validation_split = 0.2` to let TF decide which data points should be set aside for validation during the fit. See below: 

In [ ]:
basic_model_2 = tf.keras.models.Sequential()
basic_model_2.add(tf.keras.layers.Flatten())
basic_model_2.add(tf.keras.layers.Dense(100, activation = 'relu'))
basic_model_2.add(tf.keras.layers.Dense(num_classes, activation = 'softmax'))

basic_model_2.compile(optimizer = 'SGD', 
                    loss = tf.keras.losses.categorical_crossentropy,
                    metrics = ['accuracy'])

basic_model_2.fit(x = x_train, 
                y = y_train,
                validation_split = 0.2)

The validation accuracy can now be seen from the output above. This value changes everytime the code is run as different parts of the data are reserved each time for the validation.

Now would be a good time to infer each of the output terms above.



Further topics to explore include:
- Activation functions
- Loss Functions
- Epochs
- Batch sizes
- Optimisation Methods
- Node count per layer
- Number of layers
- Dropout
- Different types of layers
- Convolutional Layers
- Stride length
- Data Augmentation (not really applicable here)
